# ConvNet on MNIST

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

import sys
from twodlearn.feedforward import *

## 1. Load MNIST

In [2]:
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
from twodlearn.datasets.mnist import MnistDataset
mnist = MnistDataset('MNIST_data')


File already downloaded.
File already downloaded.
File already downloaded.
File already downloaded.
loading MNIST_data/train-images-idx3-ubyte.gz
loading MNIST_data/t10k-images-idx3-ubyte.gz


In [3]:
batch_X, batch_y = mnist.train.next_batch(100)

In [4]:
print(np.reshape(batch_X, [-1,28,28,1]).shape)
print(batch_y.shape)


(100, 28, 28, 1)
(100, 10)


## 2. Model definition

In [5]:
sess = tf.InteractiveSession()

In [6]:
input_shape= [28, 28, 1]
n_outputs=  batch_y.shape[1]

n_filters = [32, 64]   
filter_size = [[5,5], [5,5]] 
pool_size =   [[2,2], [2,2]] 
n_hidden = [1024]

batch_size= 100

In [7]:
# for dropout
drop_prob = tf.placeholder(tf.float32)

# Define ConvNet
conv_net= AlexNetClassifier(input_shape, n_outputs, 
                            n_filters, filter_size, 
                            pool_size, n_hidden, name='ConvNet')

# Define train setup
train= conv_net.evaluate()
train.loss.init()
# Optimizer.
optimizer = tf.train.AdamOptimizer(1e-3).minimize(tf.convert_to_tensor(train.loss)) #0.001
  
# Predictions for the training, validation, and test data.
train_prediction = tf.convert_to_tensor(train.output.value)

## 3. Training

In [9]:
def accuracy(predictions, labels):
    if n_outputs == 1:
        return (100.0 * np.sum(np.greater(predictions, 0.5) == np.greater(labels, 0.5))/ predictions.shape[0])
    else:
        return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

In [10]:
num_steps = 2000
summary_freq= 100
n_test_logg= 10 # number of evaluations on test dataset (for logging information)

tf.global_variables_initializer().run()
print('Initialized')

mean_loss= 0
train_accuracy= 0
for step in range(num_steps):
    batch_X, batch_y= mnist.train.next_batch(batch_size)
    batch_X= np.reshape(batch_X, [-1,28,28,1])
    
    
    feed_dict = {train.inputs : batch_X, train.loss.labels : batch_y, drop_prob:1.0}
    
    _, l, train_pred = sess.run([optimizer, tf.convert_to_tensor(train.loss), train_prediction],feed_dict=feed_dict)
    
    train_accuracy += accuracy(train_pred, batch_y)
    mean_loss += l    
    
    if step%summary_freq == 0:
        # train
        train_accuracy= train_accuracy/summary_freq
        
        # test
        test_accuracy= 0
        for i in range(n_test_logg):
            batch_X_test, batch_y_test= mnist.test.next_batch(batch_size) 
            batch_X_test= np.reshape(batch_X_test, [-1,28,28,1])

            pred = train_prediction.eval(feed_dict={train.inputs: batch_X_test, drop_prob: 1.0})
            
            test_accuracy += accuracy(pred, batch_y_test)
        test_accuracy= test_accuracy/n_test_logg
            
        print(step, ', train:',train_accuracy,' | test:', test_accuracy, ' | loss:', mean_loss/summary_freq)
        mean_loss= 0
        train_accuracy= 0

Initialized
0 , train: 0.03  | test: 13.6  | loss: 0.5980055999755859
100 , train: 77.24  | test: 91.2  | loss: 14.363180601894856
200 , train: 94.27  | test: 93.4  | loss: 0.19585871433839203
300 , train: 94.95  | test: 94.6  | loss: 0.18131906697526573
400 , train: 95.49  | test: 94.7  | loss: 0.15943107897415756
500 , train: 95.96  | test: 97.9  | loss: 0.14832110987976194
600 , train: 97.8  | test: 97.4  | loss: 0.08166376390960067
700 , train: 98.04  | test: 99.0  | loss: 0.07183787561021746
800 , train: 97.93  | test: 99.2  | loss: 0.06840142113156616
900 , train: 97.86  | test: 97.6  | loss: 0.07480014238040894
1000 , train: 97.93  | test: 98.1  | loss: 0.07748781023081391
1100 , train: 98.17  | test: 97.7  | loss: 0.06691675838083029
1200 , train: 98.95  | test: 98.0  | loss: 0.034301836416125296
1300 , train: 98.72  | test: 98.4  | loss: 0.045355648780241606
1400 , train: 98.73  | test: 97.7  | loss: 0.04469216229394078
1500 , train: 98.39  | test: 98.0  | loss: 0.057574493903

In [11]:
assert (train_accuracy>98) and ((test_accuracy>97)), "Accurary of convnet test lower than expected"